In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load your dataset
data = pd.read_csv('/content/drive/MyDrive/Timehacks/final.csv')  # Replace with your actual file path


In [ ]:
!pip install --upgrade tensorflow


In [ ]:
import os

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [ ]:
import tensorflow as tf


In [ ]:
temp = data['pm2_5']

In [ ]:
def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(temp, WINDOW_SIZE)
X1.shape, y1.shape

((7239869, 5, 1), (7239869,))

In [ ]:
from sklearn.model_selection import train_test_split
X_temp, X_test1, y_temp, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam

model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(5, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 5, 50)             10400     
                                                                 
 lstm_4 (LSTM)               (None, 50)                20200     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30651 (119.73 KB)
Trainable params: 30651 (119.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X1, y1, epochs=10, verbose=1)


Epoch 1/10
226246/226246 [==============================] - 1087s 5ms/step - loss: 98.9022
Epoch 2/10
226246/226246 [==============================] - 1096s 5ms/step - loss: 98.3985
Epoch 3/10
226246/226246 [==============================] - 1085s 5ms/step - loss: 101.3717
Epoch 4/10
226246/226246 [==============================] - 1092s 5ms/step - loss: 97.6301
Epoch 5/10
226246/226246 [==============================] - 1080s 5ms/step - loss: 101.1666
Epoch 6/10
192749/226246 [========================>.....] - ETA: 2:42 - loss: 103.9388

KeyboardInterrupt: 

In [ ]:
# Assuming you're predicting 'pm2_5' based on 'temperature', 'humidity', 'pm10'
features = data[['latitude','longitude','temperature', 'humidity', 'pm10']].values
target = data['pm2_5'].values.reshape(-1, 1)

# Normalize features and target
scaler_features = MinMaxScaler(feature_range=(0, 1))
scaler_target = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler_features.fit_transform(features)
scaled_target = scaler_target.fit_transform(target)


In [ ]:
def create_sequences(features, target, steps):
    X, y = [], []
    for i in range(len(features) - steps):
        X.append(features[i:(i + steps)])
        y.append(target[i + steps])
    return np.array(X), np.array(y)

input_steps = 20  # Adjust based on your model's design
X, y = create_sequences(scaled_features, scaled_target, input_steps)


In [ ]:
# First, split into train+validation (75%) and test (25%)
test_size = int(len(X) * 0.25)  # Calculate the number of samples for testing

# Split the data
X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
n_input = 60  # Number of time steps
n_features = 5
# Define model architecture
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(n_input, n_features)))
model.add(Dense(1))
# model = LSTMModel(input_dim=5, hidden_dim=50, layer_dim=1, output_dim=1)

model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # One time step (the last one perhaps)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

# Assuming input_dim=3 for 'temperature', 'humidity', 'pm10' features, hidden_dim=50, layer_dim=1, output_dim=1 for 'pm2_5'
model = LSTMModel(input_dim=5, hidden_dim=50, layer_dim=1, output_dim=1)


In [ ]:
# Convert arrays to PyTorch tensors
train_X_tensor = torch.Tensor(train_X)
train_y_tensor = torch.Tensor(train_y)
valid_X_tensor = torch.Tensor(valid_X)
valid_y_tensor = torch.Tensor(valid_y)

loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 100  # Adjust based on convergence
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_X_tensor)
    loss = loss_function(outputs, train_y_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print every 10th epoch
        model.eval()
        valid_outputs = model(valid_X_tensor)
        valid_loss = loss_function(valid_outputs, valid_y_tensor)
        print(f'Epoch {epoch}, Training Loss: {loss.item()}, Validation Loss: {valid_loss.item()}')


In [ ]:
model.eval()
test_X_tensor = torch.Tensor(X_test)
test_y_tensor = torch.Tensor(y_test)
predictions = model(test_X_tensor)
test_loss = loss_function(predictions, test_y_tensor)
print(f'Test Loss: {test_loss.item()}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the LSTM model with reduced complexity
class SimplifiedLSTM(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=64, output_size=1):
        super(SimplifiedLSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)

        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Assuming we have 3 input features and 1 output
model = SimplifiedLSTM(input_size=3, hidden_layer_size=64, output_size=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Assuming train_X, train_y, valid_X, valid_y are already defined and properly scaled
# Convert the datasets to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32)
train_y_tensor = torch.tensor(train_y, dtype=torch.float32).view(-1, 1)
valid_X_tensor = torch.tensor(valid_X, dtype=torch.float32)
valid_y_tensor = torch.tensor(valid_y, dtype=torch.float32).view(-1, 1)

# Training loop
epochs = 150  # or a suitable number of epochs
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(train_X_tensor)
    loss = criterion(output, train_y_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Evaluation every 10 epochs
        model.eval()
        with torch.no_grad():
            valid_output = model(valid_X_tensor)
            valid_loss = criterion(valid_output, valid_y_tensor)
        print(f'Epoch {epoch}, Loss: {loss.item()}, Validation Loss: {valid_loss.item()}')

# Remember to adjust input_size, hidden_layer_size, and output_size according to your dataset specifics.
